<a href="https://colab.research.google.com/github/scottmarino-io/Trading/blob/main/OptionsDeviationCalc_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta, date

# Input Validation
try:
    print('Enter the ticker symbol to analyze:')
    ticker = input().strip().upper()
    if not ticker:
        raise ValueError("Ticker symbol cannot be empty.")

    print('Enter the option expiration date (YYYY-MM-DD):')
    chose_expiry_date = input().strip()
    expiry_date = datetime.strptime(chose_expiry_date, '%Y-%m-%d').date()
    if expiry_date < date.today():
        raise ValueError("Expiration date cannot be in the past.")

    print('Enter the timeframe to compare against must be one of 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max')
    date = input().strip().upper()
    if not date:
        raise ValueError("Date cannot be empty.")

    print('Enter the data interval must be one of 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo')
    interval = input().strip().upper()
    if not date:
        raise ValueError("Date cannot be empty.")

except Exception as e:
    print(f"Input error: {e}")
    exit()

# Validate Ticker Symbol
try:
    yf_info = yf.Ticker(ticker)
    if yf_info.history(period="1d").empty:
        raise ValueError("Ticker not found or invalid ticker symbol.")
    print(f"Valid ticker: {ticker}")
except Exception as e:
    print(f"Error validating ticker: {e}")
    exit()

# Utility Functions
def nearest_value(input_list, find_value):
    return min(input_list, key=lambda x: abs(x - find_value))

def weekdays_calculator(end_str):
    today = datetime.today().date()
    end = datetime.strptime(end_str, '%Y-%m-%d').date()
    return np.busday_count(today, end)

def split_list(input_list, n):
    for i in range(0, len(input_list), n):
        yield input_list[i:i + n]

def yf_info(ticker):
    return yf.Ticker(ticker)

def current_price(ticker):
    try:
        return yf_info(ticker).info['bid']
    except KeyError:
        raise ValueError("Market price not available for the ticker.")

def historical_data(ticker, period=date, interval=interval):
    try:
        hist_price = yf.download(ticker, period=period, interval=interval, auto_adjust=True)[['Close']]
        if hist_price.empty:
            raise ValueError("No historical data available.")
        return hist_price
    except Exception as e:
        raise ValueError(f"Error fetching historical data: {e}")

def perc_change(ticker, horizon):
    data = historical_data(ticker)
    horizon = horizon - 1
    data['return_perc'] = data['Close'].pct_change(periods=horizon, fill_method='ffill').round(4)
    return data.dropna()

def latest_perc_change(ticker, horizon, past_days):
    historical = perc_change(ticker, horizon)
    return historical['return_perc'].values[-past_days:]

def describe_perc_change(ticker, horizon):
    cur_price = current_price(ticker)
    data = perc_change(ticker, horizon)
    describe = data.describe(percentiles=np.linspace(0, 1, 101))
    describe['Close'] = cur_price
    describe['price'] = cur_price * (1 + describe['return_perc'])
    describe['return_perc'] = describe['return_perc'] * 100
    describe['price_int'] = describe['price'].astype(int)
    return describe

def option_expiry_dates(ticker):
    try:
        return yf_info(ticker).options
    except Exception as e:
        raise ValueError(f"Error fetching option expiry dates: {e}")

def option_chain(ticker, expiry_date, call_or_put=None, in_or_out=None):
    try:
        result = yf_info(ticker).option_chain(expiry_date)
        if call_or_put is None:
            call = result.calls
            put = result.puts
            result = pd.concat([call, put], ignore_index=True)
        elif call_or_put in ['call', 'put']:
            result = result.calls if call_or_put == 'call' else result.puts
        else:
            raise ValueError("Invalid value for call_or_put. Use 'call' or 'put'.")

        if in_or_out == 'in':
            result = result.loc[result['inTheMoney']]
        elif in_or_out == 'out':
            result = result.loc[~result['inTheMoney']]
        return result
    except Exception as e:
        raise ValueError(f"Error fetching option chain data: {e}")

def perc_change_with_option(ticker, horizon, expiry_date, call_or_put=None, in_or_out=None):
    perc_change_data = describe_perc_change(ticker, horizon)
    price_int_l = perc_change_data['price_int'].values.tolist()
    opt_data = option_chain(ticker, expiry_date, call_or_put, in_or_out)
    all_strikes = opt_data.strike.tolist()

    chose_strike = []
    last_price = []
    for i in price_int_l:
        nearest_strike = nearest_value(all_strikes, i)
        lp = opt_data.loc[opt_data['strike'] == nearest_strike].lastPrice.values
        chose_strike.append(int(nearest_strike))
        last_price.append(lp[0] if len(lp) > 0 else None)

    perc_change_data['chose_strike'] = chose_strike
    perc_change_data['last_price'] = last_price
    perc_change_data.loc['count', 'return_perc'] /= 100
    return perc_change_data

def plot_histogram(ticker, horizon, latest_horizon):
    historical = perc_change(ticker, horizon)
    all_perc_change_l = historical['return_perc'].values.tolist()
    latest_perc_change_l = all_perc_change_l[-latest_horizon:]
    cur_per_change = all_perc_change_l[-1:]

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=all_perc_change_l, name='All Records', opacity=0.75))
    fig.add_trace(go.Histogram(x=latest_perc_change_l, name=f'Latest {latest_horizon} Trading Days', opacity=0.9))
    fig.add_trace(go.Histogram(x=cur_per_change, name='Current', opacity=1))
    fig.update_layout(
        barmode='overlay',
        title=f'% Change Distribution Count - ({horizon} days)',
        xaxis_title='% Change',
        yaxis_title='Count'
    )
    fig.show()

# Main Execution
try:
    days_to_expiry = weekdays_calculator(chose_expiry_date)
    print(perc_change_with_option(ticker, days_to_expiry, chose_expiry_date, call_or_put=None, in_or_out='out'))
    plot_histogram(ticker, days_to_expiry, 20)
except Exception as e:
    print(f"Error during execution: {e}")

Enter the ticker symbol to analyze:
SPY
Enter the option expiration date (YYYY-MM-DD):
2025-01-17
Enter the timeframe to compare against must be one of 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
10y
Enter the data interval must be one of 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
1d
Valid ticker: SPY


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-e222b41034a8>:78: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['return_perc'] = data['Close'].pct_change(periods=horizon, fill_method='ffill').round(4)
[*********************100%***********************]  1 of 1 completed

Price    Close return_perc         price price_int chose_strike last_price
Ticker     SPY                                                            
count   592.04         NaN  1.486020e+06   1486020          840       0.01
mean    592.04    0.439861  5.946442e+02       594          594       5.16
std     592.04    2.827559  6.087803e+02       608          608       0.63
min     592.04  -23.340000  4.538579e+02       453          455       0.07
0%      592.04  -23.340000  4.538579e+02       453          455       0.07
...        ...         ...           ...       ...          ...        ...
97%     592.04    4.970000  6.214644e+02       621          621       0.05
98%     592.04    5.632400  6.253861e+02       625          625       0.03
99%     592.04    6.816000  6.323934e+02       632          630       0.02
100%    592.04   15.290000  6.825629e+02       682          680       0.01
max     592.04   15.290000  6.825629e+02       682          680       0.01

[106 rows x 6 columns]



<ipython-input-1-e222b41034a8>:78: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['return_perc'] = data['Close'].pct_change(periods=horizon, fill_method='ffill').round(4)
